In [2]:
import sys
sys.dont_write_bytecode = True
from transformers import LlamaTokenizer, LlamaForCausalLM, GenerationConfig, pipeline
import transformers
import torch
%load_ext autoreload
%autoreload 2
sys.path.append('src/')
import transformers 
import torch
from util import unshard_gedeepspeed

def generate_text(model: torch.nn.Module, tokenizer, text):
    
    input_ids = tokenizer(text, return_tensors="pt")["input_ids"]

    generation_config = transformers.GenerationConfig(
        max_new_tokens=200,
        temperature=0.6,
        top_p=0.95,
        repetition_penalty=1.2
    )

    base_model.eval()

    with torch.no_grad():

        outputs = model.generate(
            input_ids=input_ids,
            generation_config=generation_config,
            return_dict_in_generate=True,
            output_scores=True,
            pad_token_id=tokenizer.pad_token_id,
        )

    response = tokenizer.decode(outputs.sequences[0], skip_special_tokens=True)
    
    return response

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [ ]:
from reacher.reacher import Reacher, ReacherDocker, RemoteClient

reacher = Reacher(
    build_name="training",
    host="sshc.jarvislabs.ai",
    port=18960,
    user="root",
    ssh_key_filepath="/home/johannes/.ssh/id_rsa.pub",
)

reacher.setup()

In [ ]:
%%writefile test.py

import time 
for i in range(10000):
    print(i)
    time.sleep(1)

In [ ]:
reacher.put("test2.py")
reacher.execute_command("python3 test2.py", wrap_in_screen=True, named_session="test")

In [ ]:
reacher.list_named_sessions()

In [ ]:
reacher.kill_named_session("test")

## Training

### Send all data to the remote

In [ ]:
reacher.put(["src", "data", "config.yaml"])

In [ ]:
# Process the data
reacher.execute_command(
    "python3 src/setup.py --local --model llama --data_file data/alpaca_data_cleaned.json --max_tokens 512",
    wrap_in_screen=True,
    named_session="test",
)

In [ ]:
# Train the model
reacher.execute_command(
    "python3 src/train_lightning.py -c config.yaml ",
    wrap_in_screen=True,
    named_session="test",
)

In [ ]:
# Check the different runs
reacher.ls("artifacts/alpaca/")

In [ ]:
reacher.get()

In [ ]:
CHECKPOINT_PATH = ""

## Load the checkpointed model

In [ ]:
from util import unshard_deepspeed, TokenizerTokens
from model import GeneratorModel, Llama

unshard_deepspeed(CHECKPOINT_PATH, "final.ckpt")

checkpoint_model = GeneratorModel.load_from_checkpoint(
    "final.ckpt",
    base_model=Llama(from_pretrained=True, lora=True),
    strict=False
)

model = checkpoint_model._model

tokenizer = transformers.LlamaTokenizer.from_pretrained(
    "decapoda-research/llama-7b-hf"
)

tokenizer.add_special_tokens(
    {
        "eos_token": TokenizerTokens.DEFAULT_EOS_TOKEN,
        "bos_token": TokenizerTokens.DEFAULT_BOS_TOKEN,
        "unk_token": TokenizerTokens.DEFAULT_UNK_TOKEN,
        "pad_token": TokenizerTokens.DEFAULT_PAD_TOKEN,
    }
)

# model.half().to("cuda")

In [ ]:
from util import unshard_deepspeed
from model import GeneratorModel, Gpt2

checkpoint_model = GeneratorModel.load_from_checkpoint(
    "final.ckpt",
    base_model=Gpt2(from_pretrained=True, lora=True),
    strict=True
)

model = checkpoint_model._model

tokenizer = transformers.GPT2Tokenizer.from_pretrained("gpt2-large")

tokenizer.pad_token = tokenizer.bos_token 

# model.half().to("cuda")

## Generate Text

In [ ]:
from util import generate_prompt
text = generate_prompt(
    input="",
    instruction=""
)
generated_text = generate_text(model=model, tokenizer=tokenizer, text=text)

In [ ]:
generated_text